# From the address to the 3D

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import geopy as gp
import folium

import fiona

from shapely.geometry import Point, Polygon

import rasterio as rio
from rasterio.plot import show
from rasterio.windows import Window

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import open3d as o3d

In [2]:
from functions import *
from raster2xyz import raster2xyz

## Enter an address, get a location

In [3]:
# Google API

country_code = -1

while country_code != 'BE':
    address = input("Please, enter an address in Begium:\n")

    locator = gp.geocoders.GoogleV3(api_key='')
    location = locator.geocode(address, timeout=10)

    postcode = int(location.raw['address_components'][-1]['long_name'])
    country_code = location.raw['address_components'][-2]['short_name']
    coord = (location.latitude, location.longitude)

print("Latitude = {}, Longitude = {}, Postcode = {}".format(coord[1], coord[0], postcode))

Please, enter an address in Begium:
Bois du cazier
Latitude = 4.444325200000001, Longitude = 50.3811776, Postcode = 6001


#### OpenStreet Map (gratuit, moins efficace)

country_code = -1

while country_code != 'be':
    address = input("Please, enter an address in Begium:\n")

    locator = gp.Nominatim(user_agent="myGeocoder")
    location = locator.geocode(address, addressdetails=True)
    
    postcode = int(location.raw['address']['postcode'])
    country_code = location.raw['address']['country_code']
    coord = (location.latitude, location.longitude)

print("Latitude = {}, Longitude = {}, Postcode = {}".format(coord[0], coord[1], postcode))

In [4]:
mappy = folium.Map(
    location=[coord[0],coord[1]],
    zoom_start=17
)

folium.CircleMarker(
    location=[coord[0],coord[1]],
    radius=30,
    popup='Your address',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(mappy)

mappy

## From coordinates to correct crs

In [5]:
p = gpd.GeoSeries([Point(coord[1], coord[0])])
p.crs = 'epsg:4326'
p = p.to_crs(epsg=31370)
print(p[0])

x = p[0].x
y = p[0].y

POINT (155375.4713449882 118920.0424418319)


## Get Polygons of houses and terrain coordinates

In [6]:
properties_src = fiona.open("/home/demes/Documents/Ressources/Cadastre/Bpn_CaPa_WAL.shp", "r")

In [ ]:
shape_property = []
property_area = 0
for elem in properties_src :
    if Polygon(elem["geometry"]['coordinates'][0]).contains(p[0]):
        shape_property = [elem["geometry"]]
        property_area = elem['properties']['Shape_area']
        break

In [ ]:
print(f"Area = {property_area} \nShape = {shape_property}")

buildings_src = fiona.open("/home/demes/Documents/Ressources/Cadastre/Bpn_CaBu.shp", "r")

shape_buildings = [(elem["geometry"], elem['properties']['Shape_area']) 
                   for elem in buildings_src 
                   if Polygon(shape_property[0]['coordinates'][0]).contains(
                       Point(elem["geometry"]['coordinates'][0][0]))]

for building in shape_buildings :
    print(f"Area = {building[1]} \nShape = {building[0]}")

## Get the corrects LiDAR files' path

In [ ]:
path_dsm = ""
path_dtm = ""
flanders = False

if postcode >= 1300 and postcode < 1500:
    path_dsm = set_path_dsm('BRABANT_WALLON')
    path_dtm = set_path_dtm('BRABANT_WALLON')
    print('Walloon Brabant')
    
elif postcode >= 4000 and postcode < 5000:
    path_dsm = set_path_dsm('LIEGE')
    path_dtm = set_path_dtm('LIEGE')
    print('Liège')
    
elif postcode >= 5000 and postcode < 6000:
    path_dsm = set_path_dsm('NAMUR')
    path_dtm = set_path_dtm('NAMUR')
    print('Namur')
    
elif (postcode >= 6000 and postcode < 6600) or (postcode >= 7000 and postcode < 8000):
    path_dsm = set_path_dsm('HAINAUT')
    path_dtm = set_path_dtm('HAINAUT')
    print('Hainaut')
    
elif postcode >= 6600 and postcode < 7000:
    path_dsm = set_path_dsm('LUXEMBOURG')
    path_dtm = set_path_dtm('LUXEMBOURG')
    print('Luxembourg')
    
else:
    number = [x for x in range(1, 44)]
    # ouvrir le .shp avc fiona et chercher le point (A FAIRE)
    for i in number :
        path_dsm = f'/home/demes/Documents/Resources/Flandre/DSM/DHMVIIDSMRAS1m_k{i:02}/GeoTIFF/DHMVIIDSMRAS1m_k{i:02}.tif'
        with rio.open(path_dsm) as dsm:
            #coordinates = (
             #   (x+100, y+100), (x-100, y-100)
            #)
            coordinates = (
                (p[0].x, p[0].y),
            )
            for i, (lon, lat) in enumerate(coordinates):
                # Get pixel coordinates from map coordinates
                print(lon, lat)
                py, px = dsm.index(lon, lat)
                print('Pixel Y, X coords: {}, {}'.format(py, px))

                # Build an NxN window
                small_dsm = dsm.read(1, window=Window(px - N//2, py - N//2, N, N), masked=True)
    flanders = True
    print('Brussels or Flanders region')

## Get a sample of the LiDARs, center on the coordinates

[Highlight on "no value's data"](https://github.com/Demesmaeker/3D_Houses/blob/main/App/highlight_on_no_values_data.ipynb)

##### DSM

In [ ]:
coordinates = (
    (p[0].x, p[0].y),
)

# Window's size
N = 300

print(path_dsm)

with rio.open(path_dsm) as dsm:
    for i, (lon, lat) in enumerate(coordinates):
        # Get pixel coordinates from map coordinates
        py, px = dsm.index(lon, lat)
        print('Pixel Y, X coords: {}, {}'.format(py, px))
        
        # Build an NxN window
        small_dsm = dsm.read(1, window=Window(px - N//2, py - N//2, N, N), masked=True)
        dsm_meta = dsm.profile

plt.figure(figsize=(10, 8))
plt.imshow(small_dsm)
plt.colorbar()
plt.show()

##### DTM

In [ ]:
with rio.open(path_dtm) as dtm:
    for i, (lon, lat) in enumerate(coordinates):
        # Get pixel coordinates from map coordinates
        py, px = dtm.index(lon, lat)
        print('Pixel Y, X coords: {}, {}'.format(py, px))
        
        # Build an NxN window
        small_dtm = dtm.read(1, window=Window(px - N//2, py - N//2, N, N), masked=True)
        dtm_meta = dtm.profile

plt.figure(figsize=(10, 8))
plt.imshow(small_dtm)
plt.colorbar()
plt.show()

## Canopy Height Model

[Highlight on Canopy Height Model and Hillshade](https://github.com/Demesmaeker/3D_Houses/blob/main/App/Highlight%20on%20the%20Canopy%20Height%20Model%20and%20Hillshade.ipynb)

## Transform raster into xyz dataframes

In [ ]:
shape = [{'type': 'Polygon', 
          'coordinates': [[(x+75, y+75), 
                           (x-75, y+75), 
                           (x-75, y-75), 
                           (x+75, y-75)]]}]

In [ ]:
# shape_property

buildings = []
for building in shape_buildings :
    buildings += building[0]

Get the xyz coordinates of the points

In [ ]:
base_dsm = raster2xyz.translate_from_cropped(get_mask(path_dsm, shape))
base_dtm = raster2xyz.translate_from_cropped(get_mask(path_dtm, shape))
property_dsm = raster2xyz.translate_from_cropped(get_mask(path_dsm, shape_property))
property_dtm = raster2xyz.translate_from_cropped(get_mask(path_dtm, shape_property))

Get some dataframes

In [ ]:
df_base_dsm = get_dataframe(base_dsm)
df_base_dtm = get_dataframe(base_dtm)
df_base_chm = get_chm_df(df_base_dsm, df_base_dtm)

df_property_dsm = get_dataframe(property_dsm)
df_property_dtm = get_dataframe(property_dtm)
df_property_chm = get_chm_df(df_property_dsm, df_property_dtm)

In [ ]:
df_base_dsm.describe()

In [ ]:
df_base_dsm

## Convert the datasets to 3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

X = df_property_dsm['x']
Y = df_property_dsm['y']
Z = df_property_dsm['z']

ax.scatter(X, Y, Z, c='r', marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')

plt.show()

### Points cloud

##### DSM

In [ ]:
show_pcd(df_base_dsm)

In [ ]:
show_pcd(df_property_dsm)

##### DTM

In [ ]:
show_pcd(df_base_dtm)

In [ ]:
show_pcd(df_property_dtm)

##### CHM

In [ ]:
show_pcd(df_base_chm)

In [ ]:
show_pcd(df_property_chm)

### 3D

In [ ]:
show_poisson(df_base_dsm)

In [ ]:
show_poisson(df_property_dsm)

In [ ]:
show_poisson(df_base_chm)

In [ ]:
show_poisson(df_property_chm)